In [6]:
import sys
#   paths
ANKI_PATH = 'C:/Users/plk2033/Documents/PythonDocs/anki-master'
COLLECTION_PATH = "C:/Users/plk2033/Documents/PythonDocs/collection.anki2"

sys.path.append(ANKI_PATH)
import anki
import time
from anki import Collection
from anki.utils import intTime

In [31]:
col.close()

In [34]:
col = Collection(COLLECTION_PATH)

today = intTime() #anki function returns current integer time
nextTwenty = today + 20*60 #integer time in 20 minutes

query="select id,nid,did from cards where did = 1510787588680"
result = col.db.all(query)

print(result)

col.close()

[(1482373816798, 1482373816781, 1510787588680), (1482373816799, 1482373816782, 1510787588680), (1482373816800, 1482373816783, 1510787588680), (1482373816801, 1482373816784, 1510787588680), (1482373816803, 1482373816786, 1510787588680), (1482373816804, 1482373816787, 1510787588680), (1482373816805, 1482373816788, 1510787588680), (1482894085461, 1482894085458, 1510787588680), (1482894085463, 1482894085460, 1510787588680), (1483064317672, 1483064317670, 1510787588680), (1483064317673, 1483064317671, 1510787588680), (1483235313024, 1483235313021, 1510787588680), (1483235466439, 1483235466423, 1510787588680), (1483235466440, 1483235466424, 1510787588680), (1483235466441, 1483235466425, 1510787588680), (1483235466442, 1483235466426, 1510787588680), (1483235466443, 1483235466427, 1510787588680), (1483235466444, 1483235466428, 1510787588680), (1483235466445, 1483235466429, 1510787588680), (1483235466446, 1483235466430, 1510787588680), (1483235466447, 1483235466431, 1510787588680), (14832354664

In [12]:
dID = col.decks.id('Exam 7')
dID

1510787588680

In [28]:
anki.cards.did == '1510787588680'

True

In [25]:
query="select count(id) from cards where queue = 1 and due < %s" % nextTwenty

['Ch23',
 'Ch6',
 'Patrik',
 'Sahasrabuddhe',
 'Cummins',
 'Ch7',
 'Teng&Perkins',
 'marked',
 'Source',
 'ISO',
 'Coval',
 'CATBonds',
 'Capital',
 'Panning',
 'Robertson',
 'NCCI-1',
 'NCCI-2',
 'Exam9',
 'Verrall',
 'Exam',
 'Mack1994',
 'BKM',
 'Goldfarb',
 'ASOP12',
 'Bailey&Simon',
 'Bodoff',
 'RF',
 'Clark-8',
 'Exam8',
 '7',
 'RobbinUW',
 'CH9',
 'Hurlimann',
 'CH11',
 'Shapland',
 'Ch12',
 'Ch10',
 'Ch11',
 'Ch16',
 'Ch14',
 'Ch15',
 'Couret&Venter',
 'Fisher',
 'Ch1',
 'BrehmCh5',
 'Ch3',
 'BrehmCh3',
 'BrehmCh2',
 'BrehmCh1',
 'Feldblum',
 'Ch8',
 'Ch9',
 'Kreps',
 'Meyers',
 'Marshall',
 'GLM',
 'Brehm',
 'Taylor',
 'Brosius',
 'Mack2000',
 'Clark',
 'Exam7',
 'McClenahan',
 'Venter',
 'Mahler1',
 'Ch2',
 'Mango',
 'Grossi',
 'Bahnemann',
 'BrehmCh4',
 'Questions',
 'RobbinIRR',
 'Ferrari',
 'Butsic',
 'Siewert',
 'Bernegger']